In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('C:\\Users\\NEHA KL\\Desktop\\manipal\\Amazon Sale Report.csv')
df.head()

C:\Users\NEHA KL\AppData\Local\Temp\ipykernel_21524\445012404.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\NEHA KL\\Desktop\\manipal\\Amazon Sale Report.csv')


index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   

  Fulfilment ship-service-level    Style              SKU       Category Size  \
0   Merchant           Standard   SET389   SET389-KR-NP-S            Set    S   
1   Merchant           Standard  JNE3781  JNE3781-KR-XXXL          kurta  3XL   
2     Amazon          Expedited  JNE3371    JNE3371-KR-XL          kurta   XL   
3   Merchant           Standard    J0341       J0341-DR-L  Western Dress    L   
4     Amazon          Expedited  JNE3671  JNE3671-TU-XXXL            Top  3XL   

   ... currency  Amount    ship-city   ship-state  ship-postal-code  \
0  ...      INR  647.62       MUMBAI  MAHARASHTRA          400081.0   
1  ...      INR  406.00    BENGALURU    KARNATAKA          560085.0   
2  ...      INR  329.00  NAVI MUMBAI  MAHARASHTRA          410210.0   
3  ...      INR  753.33   PUDUCHERRY   PUDUCHERRY          605008.0   
4  ...      INR  574.00      CHENNAI   TAMIL NADU          600073.0   

  ship-country                                      promotion-ids    B2B  \
0           IN                                                NaN  False   
1           IN  Amazon PLCC Free-Financing Universal Merchant ...  False   
2           IN       IN Core Free Shipping 2015/04/08 23-48-5-108   True   
3           IN                                                NaN  False   
4           IN                                                NaN  False   

  fulfilled-by Unnamed: 22  
0    Easy Ship         NaN  
1    Easy Ship         NaN  
2          NaN         NaN  
3    Easy Ship         NaN  
4          NaN         NaN  

[5 rows x 23 columns]

In [3]:
# Drop irrelevant columns
df.drop(columns=['Unnamed: 22'], inplace=True)

In [4]:
# Handle missing values
df['Courier Status'].fillna('Unknown', inplace=True)
df['currency'].fillna(df['currency'].mode()[0], inplace=True)
df['Amount'].fillna(df['Amount'].median(), inplace=True)
df['ship-city'].fillna('Unknown', inplace=True)
df['ship-state'].fillna('Unknown', inplace=True)
df['ship-postal-code'].fillna('Unknown', inplace=True)
df['ship-country'].fillna('Unknown', inplace=True)
df['promotion-ids'].fillna('None', inplace=True)
df['fulfilled-by'].fillna('Unknown', inplace=True)




C:\Users\NEHA KL\AppData\Local\Temp\ipykernel_21524\3145590255.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['ship-postal-code'].fillna('Unknown', inplace=True)


In [5]:
def parse_dates(date_str):
    try:
        return pd.to_datetime(date_str, format='%m-%d-%y', errors='coerce')
    except Exception as e:
        return pd.NaT

df['Date'] = df['Date'].apply(parse_dates)

# Extract date features if needed
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop(columns=['Date'], inplace=True)

In [6]:
# Convert ship-postal-code to string to avoid conversion issues and then to category
df['ship-postal-code'] = df['ship-postal-code'].astype(str)

In [7]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Status', 'Fulfilment', 'ship-service-level', 
                    'Style', 'SKU', 'Category', 'Size', 'ASIN', 'Courier Status', 
                    'ship-city', 'ship-state', 'ship-country', 'promotion-ids', 
                    'fulfilled-by', 'currency', 'ship-postal-code', 'Order ID']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = df[col].astype(str)  # Convert to string if not already
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [8]:
df.head()

index  Order ID  Status  Fulfilment  ship-service-level  Style   SKU  \
0      0     70508       0           1                   1   1342  6992   
1      1     13615       5           1                   1    847  4436   
2      2     45301       3           0                   0    533  2633   
3      3     43682       0           1                   1    373  1767   
4      4     90099       3           0                   0    754  3837   

   Category  Size  ASIN  ...  ship-city  ship-state  ship-postal-code  \
0         5     7  5563  ...       4795          28              2477   
1         8     0  5277  ...        776          23              4857   
2         8     8   675  ...       5353          28              2671   
3         7     5  4638  ...       6140          43              5659   
4         6     0  4351  ...       1466          56              5549   

   ship-country  promotion-ids    B2B  fulfilled-by    Year  Month   Day  
0             0           5777  False             0  2022.0    4.0  30.0  
1             0           2051  False             0  2022.0    4.0  30.0  
2             0           5766   True             1  2022.0    4.0  30.0  
3             0           5777  False             0  2022.0    4.0  30.0  
4             0           5777  False             1  2022.0    4.0  30.0  

[5 rows x 24 columns]

In [9]:
# Feature Engineering: Create additional features if necessary
df['Total_Qty'] = df.groupby('SKU')['Qty'].transform('sum')

In [10]:
df.head()

index  Order ID  Status  Fulfilment  ship-service-level  Style   SKU  \
0      0     70508       0           1                   1   1342  6992   
1      1     13615       5           1                   1    847  4436   
2      2     45301       3           0                   0    533  2633   
3      3     43682       0           1                   1    373  1767   
4      4     90099       3           0                   0    754  3837   

   Category  Size  ASIN  ...  ship-state  ship-postal-code  ship-country  \
0         5     7  5563  ...          28              2477             0   
1         8     0  5277  ...          23              4857             0   
2         8     8   675  ...          28              2671             0   
3         7     5  4638  ...          43              5659             0   
4         6     0  4351  ...          56              5549             0   

   promotion-ids    B2B  fulfilled-by    Year  Month   Day  Total_Qty  
0           5777  False             0  2022.0    4.0  30.0         29  
1           2051  False             0  2022.0    4.0  30.0        100  
2           5766   True             1  2022.0    4.0  30.0         12  
3           5777  False             0  2022.0    4.0  30.0        257  
4           5777  False             1  2022.0    4.0  30.0         16  

[5 rows x 25 columns]

In [11]:
# Display a single column using bracket notation
df.Total_Qty

0          29
1         100
2          12
3         257
4          16
         ... 
128970     17
128971      3
128972      6
128973      6
128974      5
Name: Total_Qty, Length: 128975, dtype: int64

In [12]:
# Set target variable
features = ['SKU', 'Qty', 'Year', 'Month']
X = df[features]
target = 'Amount'
y = df[target]

# Convert SKU to categorical
X['SKU'] = X['SKU'].astype('category')

C:\Users\NEHA KL\AppData\Local\Temp\ipykernel_21524\2410916541.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['SKU'] = X['SKU'].astype('category')


In [13]:
from xgboost import XGBRegressor
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, enable_categorical=True)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Save the trained model
import pickle

with open('C:\\Users\\NEHA KL\\Desktop\\manipal\\xgb_model.pkl', 'wb') as file:
    pickle.dump(xgb_model, file)
    
# Save the column names
columns = X.columns.tolist()
with open('C:\\Users\\NEHA KL\\Desktop\\manipal\\columns.pkl', 'wb') as file:
    pickle.dump(columns, file)

Root Mean Squared Error (RMSE): 156.27877971313035


In [14]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 24423.05698862512


In [15]:
print(X)

         SKU  Qty    Year  Month
0       6992    0  2022.0    4.0
1       4436    1  2022.0    4.0
2       2633    1  2022.0    4.0
3       1767    0  2022.0    4.0
4       3837    1  2022.0    4.0
...      ...  ...     ...    ...
128970   424    1     NaN    NaN
128971  3441    1     NaN    NaN
128972   717    2     NaN    NaN
128973  5835    1     NaN    NaN
128974  5713    1     NaN    NaN

[128975 rows x 4 columns]


In [16]:
print(y)

0         647.62
1         406.00
2         329.00
3         753.33
4         574.00
           ...  
128970    605.00
128971    605.00
128972    605.00
128973    605.00
128974    605.00
Name: Amount, Length: 128975, dtype: float64
